In [1]:
import sys
import os
# sys.path.append(top) 把392頂層目錄加到最前面 '/home/pan/django_projects/fast_api_392'
top = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
sys.path.insert(0, top)
# sys.path

In [2]:
from apps.book.classes.bbooks import BOOKS

In [9]:
init = {'bookid': '0010550825'}
book = BOOKS(**init)

In [11]:
# book.update_errcnt = 0
await book.update_info()
# book.__dict__

update_duration = 1.6947746276855469,final_proxy=None
update_errcnt = 0


In [7]:
import pandas as pd
df = pd.DataFrame({'a': [1, 2], 'b': [3, 4]})
df

,a,b
0,1,3
1,2,4


In [11]:
a={1:None}

In [12]:
a[1]=1

In [13]:
a

{1: 1}

In [ ]:
# rtext2
a = '123{}'
a.format(456)

In [ ]:
class INFO:

    def __setattr__(self, name, value):
        self.__dict__[name] = value

In [ ]:
a = INFO()

In [ ]:
a.e = 1

In [ ]:
a.__dict__

In [ ]:
b.test

In [ ]:
object.__setattr__(b, 'test', 101)

In [ ]:
object.__getattribute__(b, 'test')